In [46]:
using Pkg
using Statistics
Pkg.add(["DataFrames", "CSV"])


   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


In [47]:
using CSV
using DataFrames

# Define full paths to your files
food_path = "/Users/jaspalkhanuja/Downloads/food.csv"
food_nutrient_path = "/Users/jaspalkhanuja/Downloads/food_nutrient.csv"
nutrient_path = "/Users/jaspalkhanuja/Downloads/nutrient.csv"

# Load CSV files
food = CSV.read(food_path, DataFrame)
food_nutrient = CSV.read(food_nutrient_path, DataFrame)
nutrient = CSV.read(nutrient_path, DataFrame)

# Preview the data
println(first(food, 5))
println(first(food_nutrient, 5))
println(first(nutrient, 5))


5×5 DataFrame
 Row │ fdc_id  data_type           description            food_category_id  publication_date 
     │ Int64   String31            String                 Int64?            String15         
─────┼───────────────────────────────────────────────────────────────────────────────────────
   1 │ 319874  sample_food         HUMMUS, SABRA CLASSIC                16  2019-04-01
   2 │ 319875  market_acquisition  HUMMUS, SABRA CLASSIC                16  2019-04-01
   3 │ 319876  market_acquisition  HUMMUS, SABRA CLASSIC                16  2019-04-01
   4 │ 319877  sub_sample_food     Hummus                               16  2019-04-01
   5 │ 319878  sub_sample_food     Hummus                               16  2019-04-01
5×11 DataFrame
 Row │ id       fdc_id  nutrient_id  amount    data_points  derivation_id  min       max       median    footnote  min_year_acquired 
     │ Int64    Int64   Int64        Float64?  Int64?       Int64?         Float64?  Float64?  Float64?  String?   Int64

In [48]:

# Rename `:id` in `nutrient` to `:nutrient_id`
rename!(nutrient, :id => :nutrient_id)

# Merge food and food_nutrient on `fdc_id`
food_with_nutrients = innerjoin(food, food_nutrient, on=:fdc_id)

# Merge food_with_nutrients with nutrient on `nutrient_id`
full_data = innerjoin(food_with_nutrients, nutrient, on=:nutrient_id)

# Select relevant columns
relevant_data = select(full_data, :description, :name, :amount, :unit_name)

# Pivot the table to have nutrients as columns
pivoted_data = unstack(
    relevant_data,
    :name,       # Column to pivot into separate columns
    :amount,     # Values to fill the columns
    combine=mean # Aggregate duplicate values by taking the mean
)

# Display a sample of the processed dataset
println("Processed Data:")
println(first(pivoted_data, 5))

# Save the processed data to a new CSV file (optional)
CSV.write("/Users/jaspalkhanuja/Downloads/processed_data.csv", pivoted_data)

Processed Data:
5×227 DataFrame
 Row │ description         unit_name  Water         Nitrogen       Total lipid (fat)  Ash            Phosphorus, P  Manganese, Mn  Potassium, K  Calcium, Ca   Sodium, Na   Magnesium, Mg  Iron, Fe       Copper, Cu      Zinc, Zn       Pantothenic acid  SFA 14:0    SFA 24:0     SFA 11:0   SFA 18:0     PUFA 22:6 n-3 (DHA)  SFA 6:0    PUFA 22:4  SFA 15:0     PUFA 20:5 n-3 (EPA)  PUFA 18:2 CLAs  PUFA 20:4    SFA 4:0    SFA 16:0    TFA 16:1 t  PUFA 18:4  SFA 22:0     PUFA 22:5 n-3 (DPA)  TFA 18:1 t   PUFA 20:3 n-9  MUFA 15:1  MUFA 18:1 c  MUFA 20:1 c  PUFA 18:3 n-3 c,c,c (ALA)  SFA 20:0     TFA 22:1 t  PUFA 22:2  PUFA 20:2 n-6 c,c  MUFA 22:1 n-9  SFA 10:0   SFA 17:0     SFA 12:0   SFA 8:0    TFA 18:2 t not further defined  MUFA 16:1 c  PUFA 20:3 n-6  TFA 18:3 t  PUFA 18:3 n-6 c,c,c  PUFA 18:2 n-6 c,c  MUFA 14:1 c  MUFA 17:1    MUFA 24:1 c  PUFA 20:3 n-3  Starch     Folate, total  Selenium, Se  Riboflavin   Niacin          Fiber, total dietary  Thiamin   Vitamin

"/Users/jaspalkhanuja/Downloads/processed_data.csv"

In [49]:
using CSV
using DataFrames

# Define file paths
input_path = "/Users/jaspalkhanuja/Downloads/processed_data.csv"
output_path = "/Users/jaspalkhanuja/Downloads/processed_data_cleaned.csv"

# Load the processed data
processed_data = CSV.read(input_path, DataFrame)

# Replace `missing` with `0` in each column
for col in names(processed_data)
    if eltype(processed_data[!, col]) == Union{Missing, T} where T
        replace!(processed_data[!, col], missing => 0)
    end
end

# Save the cleaned dataset
CSV.write(output_path, processed_data)
println("Cleaned data saved to: ", output_path)


Cleaned data saved to: /Users/jaspalkhanuja/Downloads/processed_data_cleaned.csv


In [52]:
using CSV
using DataFrames

# Load the cleaned dataset
input_path = "/Users/jaspalkhanuja/Downloads/processed_data_cleaned.csv"
cleaned_data = CSV.read(input_path, DataFrame)

# Define nutrient thresholds for various diseases
disease_nutrient_thresholds = Dict(
    "Diabetes" => Dict(
        "Vitamin D (D2 + D3)" => 5,
        "Vitamin C, total ascorbic acid" => 10,
        "Vitamin E (alpha-tocopherol)" => 2,
        "Magnesium, Mg" => 50,
        "Zinc, Zn" => 2,
        "Fiber, total dietary" => 3
    ),
    "Heart Disease" => Dict(
        "Omega-3 fatty acids" => 0.5,  # Example: EPA/DHA in grams
        "Potassium, K" => 350,
        "Magnesium, Mg" => 50,
        "Vitamin C, total ascorbic acid" => 10,
        "Fiber, total dietary" => 3
    ),
    "Osteoporosis" => Dict(
        "Calcium, Ca" => 200,
        "Vitamin D (D2 + D3)" => 5,
        "Vitamin K" => 0.1,
        "Magnesium, Mg" => 50,
        "Phosphorus, P" => 100
    ),
    # Add other diseases similarly
)

# Filter foods for each disease
function filter_foods_for_diseases(data::DataFrame, disease_thresholds::Dict)
    results = Dict()
    for (disease, thresholds) in disease_thresholds
        # Minimum number of thresholds to meet (majority)
        min_thresholds = Int(ceil(length(thresholds) / 2))
        
        # Filter rows based on nutrient thresholds
        filtered_foods = filter(row -> begin
            count_met = 0
            for (nutrient, threshold) in thresholds
                if nutrient in names(data)
                    if coalesce(row[nutrient], 0) >= threshold
                        count_met += 1
                    end
                end
            end
            count_met >= min_thresholds
        end, data)
        
        results[disease] = filtered_foods
    end
    return results
end

# Apply the function
filtered_foods_by_disease = filter_foods_for_diseases(cleaned_data, disease_nutrient_thresholds)

# Save results to CSV
output_dir = "/Users/jaspalkhanuja/Downloads"
for (disease, foods) in filtered_foods_by_disease
    output_path = joinpath(output_dir, "$disease.csv")
    CSV.write(output_path, foods)
    println("Filtered foods for $disease saved to: $output_path")
end


Filtered foods for Heart Disease saved to: /Users/jaspalkhanuja/Downloads/Heart Disease.csv
Filtered foods for Osteoporosis saved to: /Users/jaspalkhanuja/Downloads/Osteoporosis.csv
Filtered foods for Diabetes saved to: /Users/jaspalkhanuja/Downloads/Diabetes.csv
